# 2.5. Convert dimensionality reduction results to plink `.cov` format

In [3]:
import pandas as pd
import numpy as np
import os

PREFIX = os.path.join(os.environ["HOME"],"Dropbox/School/UCSD/Year2/Spring2020/CSE284/project/plink-182")
DR_OUT_PREFIX = os.path.join(PREFIX, "dr_outputs")
PLINK_DR_OUT_PREFIX = os.path.join(PREFIX, "cov_dr_outputs")

sample_metadata = pd.read_csv(os.path.join(PREFIX, "data", "sample_metadata.tsv"), sep="\t", index_col=0)

for dr_output_filename in os.listdir(DR_OUT_PREFIX):
    # Load data
    reduced_data = np.loadtxt(os.path.join(DR_OUT_PREFIX, dr_output_filename))
    reduced_data_df = pd.DataFrame(reduced_data)
    reduced_data_df.index = sample_metadata.index
    reduced_data_df.index.name = "FID"
    
    # Rename the PC columns
    num_axes = len(reduced_data_df.columns)
    axis_col_names = []
    for i in range(1, num_axes + 1):
        axis_col_names.append("Axis{}".format(i))
    reduced_data_df.columns = axis_col_names
    
    # Add on another column that's just the sample ID again, because plink expects this?
    reduced_data_df.insert(0, "IID", sample_metadata.index)
    
    reduced_data_df.to_csv(
        os.path.join(PLINK_DR_OUT_PREFIX, dr_output_filename.replace(".txt", ".cov")),
        sep="\t"
    )